In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
%run scripts/helper.py
%run scripts/models.py

In [3]:
crowd_train = load_file('./data/train.csv/train.csv', index_col='id')
# crowd_test = load_file('./data/test.csv/test.csv', index_col='id')

In [4]:
# cross validation
# train_index, test_index = ssSplit(crowd_train.median_relevance, train_size=8000, random_state=4)

In [5]:
# number of training examples
# print len(train_index)

In [6]:
# number of test examples
# print len(test_index)

In [7]:
# prepare training and validation set

# train = crowd_train.iloc[train_index]
# train_target = crowd_train.median_relevance.iloc[train_index].values
# validation = crowd_train.iloc[test_index]
# validation_target = crowd_train.median_relevance.iloc[test_index].values

In [8]:
## Prepare training on full training and test set

train = crowd_train
train_target = crowd_train.median_relevance.values
# validation = crowd_test

In [9]:
train_text = prepareText(train)
#validation_text = prepareText(validation)

### TFIDF on  full dataset rather than only on training set

In [10]:
# train_plus_test = []

# for i in range(len(train_text)):
#     train_plus_test.append(train_text[i])

# for i in range(len(validation_text)):
#     train_plus_test.append(validation_text[i])

In [11]:
# len(train_plus_test), ' ', len(train_text), ' ', len(validation_text)

### Non Linear Model using SVM

In [12]:
(non_linear_model, tfv, svd, scl) = build_non_linear_model(train_text, train_target)

In [13]:
# print weighted kappa score on the training and test set


# training_preds_non = non_linear_model_predictions(non_linear_model, tfv, svd, scl, train_text)
# validation_preds_non = non_linear_model_predictions(non_linear_model, tfv, svd, scl, validation_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_non, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_non, validation_target))

### Kneighbors model

In [14]:
# (knn_model, tfv_knn, svd_knn) = build_knn_model(train_text, train_target)

In [15]:
# training_preds_knn = knn_model_predictions(knn_model, tfv_knn, svd_knn, train_text)
# validation_preds_knn = knn_model_predictions(knn_model, tfv_knn, svd_knn, validation_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_knn, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_knn, validation_target))

### Naive Bayes Model

In [16]:
# (bayes_model, tfv_bayes) = build_naive_bayes(train_text, train_target)

In [17]:
# training_preds_bayes = naive_bayes_predictions(bayes_model, tfv_bayes, train_text)
# validation_preds_bayes = naive_bayes_predictions(bayes_model, tfv_bayes, validation_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_bayes, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_bayes, validation_target))

### Different model which tweaks stop words

In [18]:
train_tweaked_text = tweak_text(train)
# validation_tweaked_text = tweak_text(validation)

In [19]:
# train_plus_test_tweak = []

# for i in range(len(train_tweaked_text)):
#     train_plus_test_tweak.append(train_tweaked_text[i])

# for i in range(len(validation_tweaked_text)):
#     train_plus_test_tweak.append(validation_tweaked_text[i])

In [20]:
(tweaked_model, tfv_tweaked, svd_tweaked, scl_tweaked) = build_stopwords_tweak_model(train_tweaked_text, train_target)

In [21]:
# training_preds_tweak = non_linear_model_predictions(tweaked_model, tfv_tweaked, svd_tweaked, scl_tweaked, train_tweaked_text)
# validation_preds_tweak = non_linear_model_predictions(tweaked_model, tfv_tweaked, svd_tweaked, scl_tweaked, validation_tweaked_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_tweak, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_tweak, validation_target))

### Read big test set csv file in chunks of 5000 rows

In [22]:
test_predictions = []
test_ids = []

In [23]:
for test in pd.read_csv('./data/test.csv/test.csv', index_col='id', chunksize=5000):
    test = test.fillna('')
    
    test_text = prepareText(test)
    test_preds_non = non_linear_model_predictions(non_linear_model, tfv, svd, scl, test_text)

    test_tweaked_text = tweak_text(test)
    test_preds_tweaked = non_linear_model_predictions(tweaked_model, tfv_tweaked, svd_tweaked, scl_tweaked, test_tweaked_text)

    test_predictions.append((test_preds_non + test_preds_tweaked) / 2)
    test_ids.append(test.index.values.astype(int))
    

In [24]:
preds = np.hstack(test_predictions)
ids = np.hstack(test_ids)

### Lets see how the ensemble of these different techniques work

In [25]:
# pred_train = (training_preds_tweak + training_preds_non ) / 2
# pred_test = (validation_preds_tweak + validation_preds_non ) / 2

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(pred_train, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(pred_test, validation_target))

In [27]:
make_submission(ids, preds, 'ensemble_high_hopes_fingers_crossed.csv')